In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
from pandas.tseries.offsets import MonthEnd
import datetime
from tqdm import tqdm
from argparse import ArgumentParser, Namespace

In [ ]:
path = r"C:\Users\Mathiass\OneDrive - Universität Zürich UZH\Documents\mt_literature\data"
path = Path(path)

In [ ]:
# raw option return data
raw = pd.read_parquet(path/"sp500_op_ret.parquet")

In [ ]:
raw

In [ ]:
# dates to datetime
raw["date"] = pd.to_datetime(raw["date"])
raw["exdate"] = pd.to_datetime(raw["exdate"])
raw["date"] = raw["date"] + MonthEnd(0)

In [ ]:
len(raw)

In [ ]:
raw_call = raw[raw["cp_flag"] == "C"].copy()

In [ ]:
# raw_call

In [ ]:
# raw_call["optionid"].tolist()

In [ ]:
# raw_call[raw_call["optionid"] == 10980421]

In [ ]:
# def opt_ret3(df):
#     for opt_id in tqdm(df["optionid"].tolist()):
#         df_opt = df[df["optionid"] == opt_id]
#         for i, idx in enumerate(df_opt.index):
#             if i + 1 == len(df_opt.index):
#                 df.loc[idx, "OPT_RET"] = None
#                 break
#             if df_opt.iloc[i]["date"] + MonthEnd(1) == df_opt.iloc[i+1]["date"]:
#                 t1 = (df_opt.iloc[i]["delta"] * df_opt.iloc[i+1]["spotprice"]) - df_opt.iloc[i+1]["mid_price"]
#                 t = (df_opt.iloc[i]["delta"]  * df_opt.iloc[i]["spotprice"]) - df_opt.iloc[i]["mid_price"]
#                 df.loc[idx, "OPT_RET"] = t1/t - 1
#             else:
#                 df.loc[idx, "OPT_RET"] = None
#     return df

In [ ]:
# l = [10980421, 11618802, 10167707]

In [ ]:
# raw_call[np.isin(raw_call["optionid"], l)]

In [ ]:
# opt_ret3(raw_call[np.isin(raw_call["optionid"], l)])

In [ ]:
# opt_ret3(raw_call)

In [ ]:
# def count_succ(dates):
#     counter = 0
#     current = pd.to_datetime("1900") #startdate outside of dataset
#     for date in dates:
#         if date == current + MonthEnd(1):
#             counter += 1
#         current = date
#     return counter

In [ ]:
optid_counts = raw_call.groupby("optionid")["secid"].agg("count")

In [ ]:
# only optionids with count > 1 can potentially have a return
atleasttwo = optid_counts[optid_counts > 1].index.tolist()

In [ ]:
# select only rows that have more than 1 options per optionid
# raw_call_2 = raw_call[np.isin(raw_call["optionid"], atleasttwo)]

In [ ]:
# import time
# st_time = time.time()
# res = raw_call_2.groupby("optionid")["date"].agg(count_succ)
# e_time = time.time()
# print(e_time - st_time, "seconds")

In [ ]:
# Number of returns that can be used as dataset:
# sum(res[res > 0])

In [ ]:
# atleast1succ = res[res > 0].index.tolist()

In [ ]:
# def opt_ret2(df, atleast1succ):
#     for opt_id in tqdm(atleast1succ):
#         df_opt = df[df["optionid"] == opt_id]
#         for i, idx in enumerate(df_opt.index):
#             if i + 1 == len(df_opt.index):
#                 df.loc[idx, "OPT_RET"] = None
#                 break
#             if df_opt.iloc[i]["date"] + MonthEnd(1) == df_opt.iloc[i+1]["date"]:
#                 t1 = (df_opt.iloc[i]["delta"] * df_opt.iloc[i+1]["spotprice"]) - df_opt.iloc[i+1]["mid_price"]
#                 t = (df_opt.iloc[i]["delta"]  * df_opt.iloc[i]["spotprice"]) - df_opt.iloc[i]["mid_price"]
#                 df.loc[idx, "OPT_RET"] = t1/t - 1
#             else:
#                 df.loc[idx, "OPT_RET"] = None
#     return df

In [ ]:
raw_call_final = opt_ret2(raw_call, atleast1succ)

In [ ]:
# raw_call_final.to_parquet(path/"sp500_call_cao2.parquet")

In [ ]:
# raw_call_final_dropna = raw_call_final.dropna(axis=0)

In [ ]:
# raw_call_final_dropna.to_parquet(path/"sp500_call_cao_nadropped.parquet")

In [ ]:
# parser = ArgumentParser(description="Loads 'sp500_opt_ret.parquet', 'mapping_table.csv'" 
# "and 'datashare.parquet' files. Merges them according to the "
# "specified final target datasize 'size' and fills NaN values of the Gu "
# "dataset according to the specified function.")
# path = Path.cwd()/"data"

# parser.add_argument("size", type=str, default="small", 
#                     help="Size of the desired final dataset.")
# parser.add_argument("--path", type=str, 
#                 default=path,
#                 help="Path where all the three required data files should lie.")
# parser.add_argument("--fill_fn", type=str, default="mean", choices=["mean", "median"],
#                     help="whether mean or median should be used for filling NaN values")
# parser.add_argument("--fill_na", action="store_true", help="fill na values "
#                     "of the Gu2020 dataset.")
# parser.add_argument('--no-nafill', dest='fill_na', action='store_false',
#                     help="dont fill na values of Gu2020 dataset.")
# parser.set_defaults(fill_na=True)
# parser.add_argument("--tag", type=str, default="", help="tag to add to final df parquet files")

# args = parser.parse_args()

import argparse

args = argparse.Namespace(size="small", path=path, fill_fn="mean", fill_na=True, tag='')

In [ ]:

def prepare_dataset(args):
    # Convert string path to pathlib path.
    path = Path(args.path)
    # Check whether required files exist in the path.
    assert (path/"sp500_call_cao_nadropped.parquet").exists(), (
            "'sp500_opt_ret.parquet' does not exist in data path. "
            "If only .csv file is available, convert first using the helper "
            "function 'csv_to_parquet'."
            )
    assert (path/"mapping_table.csv").exists(), ("'mapping_table.csv' does not exist "
                                                "in data path")
    assert (path/"Gu2020_datashare\datashare.parquet").exists(), (
        "'datashare.parquet' does not exist in data path. "
        "If only .csv file is available, convert first using the helper "
        "function 'csv_to_parquet'.")

    # Load S&P 500 option returns.
    print("Read in S&P500 option returns from parquet file...")
    s_time = time.time()
    sp500opt_df = pd.read_parquet(path/"sp500_call_cao_nadropped.parquet")
    e_time = time.time()
    print("Done! Read 'sp500_call_cao_nadropped.parquet' file in:", (e_time-s_time), "seconds.")

    sp500opt_df["date"] = pd.to_datetime(sp500opt_df["date"])
    sp500opt_df["exdate"] = pd.to_datetime(sp500opt_df["exdate"])
    # Bring dates to end of month dates (in order to correctly merge with other datasets).
    sp500opt_df["date"] = sp500opt_df["date"] + MonthEnd(0)
    # Remove 138 NaN option returns.
    sp500opt_df = sp500opt_df.drop(sp500opt_df[sp500opt_df["option_ret"].isnull()].index)
    sp500opt_df = sp500opt_df.reset_index(drop=True)

    # Load mapping table data (options <-> underlying stocks). Note: The last
    # 7 lines (line 28278-28284) in the .csv. were added manually to fix minor
    # date gaps.
    print("Read in mapping table that maps S&P 500 option returns with underlying stocks...")
    s_time = time.time()
    map_table_df = pd.read_csv(path/"mapping_table.csv") # file is small enough to use .csv
    e_time = time.time()
    print("Done! Read 'mapping_table.csv' in:", (e_time-s_time), "seconds.")

    # Merge Option Returns and mapping table on "secid".
    print("Merge S&P 500 option returns with underlying stock 'secid'...")
    opt_df = sp500opt_df.merge(map_table_df, on="secid")
    print("Done!")

    # Correct format important here, otherwise will see month as day and vice versa.
    print("Filter out dates that do not lie within start and end dates...")
    opt_df["sdate"] = pd.to_datetime(opt_df["sdate"], format = "%d/%m/%Y %H:%M")
    opt_df["edate"] = pd.to_datetime(opt_df["edate"], format = "%d/%m/%Y %H:%M")

    # Filter opt_df to only contain dates that are within 'sdate' and 'edate'.
    # 'date' is used instead of 'exdate', more rows are preserved -> seems more logical.
    opt_df = opt_df[(opt_df["date"] >= opt_df["sdate"]) & 
                    ((opt_df["date"] <= opt_df["edate"]) | 
                    (opt_df["edate"] >= "2020-12-31"))]
    print(f"Done! The merged df after filtering has {len(sp500opt_df)-len(opt_df)} "
        "rows less than the original S&P 500 option return dataset.")
    # 4 observations are dropped because their date is after the 'edate' of the 
    # mapping list...

    # Load Gu2020 stock data.
    print("Load Gu2020 data...")
    s_time = time.time()
    gu2020_df = pd.read_parquet(path/"Gu2020_datashare\datashare.parquet")
    e_time = time.time()
    print("Done! Read in:", (e_time-s_time), "seconds.")

    # Lag features by -1 to align with option features. See readme file of Gu2020
    # datashare directory. They aligned their data by pushing features forward.
    # Now, we have to push them back again, since in our data the return was lagged.
    print("Clean Gu2020 data...")
    gu2020_df["DATE"] = pd.to_datetime(gu2020_df["DATE"].astype(str), 
                                        format='%Y%m%d') - MonthEnd(1)

    # Adds missing sic codes manually, after verifying them with given sources.
    gu2020_df = add_sic_manually(gu2020_df)

    # Our data starts at 1996-01-31.
    gu2020_df = gu2020_df[gu2020_df["DATE"] > "1995-12-31"]
    gu2020_df = gu2020_df.reset_index()
    gu2020_df = gu2020_df.rename(columns={"DATE": "date"})

    if args.fill_na:
        # Fill missing values.
        print(f"Fill missing values... There are {gu2020_df.isnull().to_numpy().sum()} "
                f"missing values in the Gu2020 dataset. Fill them with {args.fill_fn} of stocks of "
                "respective month...")
        # Unique set of dates to loop over and take mean/median of all stocks in that month.
        date_unique = list(gu2020_df["date"].unique())
        # Loop over each month and fill NA with stock mean/median of that month.
        if args.fill_fn == "mean":
            for i in tqdm(date_unique):
                gu2020_df[gu2020_df["date"] == i] = (gu2020_df[gu2020_df["date"] == i]
                                                    .fillna(gu2020_df[gu2020_df["date"] == i]
                                                    .mean(axis=0, numeric_only=True)))
        elif args.fill_fn == "median":
            for i in tqdm(date_unique):
                gu2020_df[gu2020_df["date"] == i] = (gu2020_df[gu2020_df["date"] == i]
                                                    .fillna(gu2020_df[gu2020_df["date"] == i]
                                                    .median(axis=0, numeric_only=True)))
        else:
            raise ValueError("Select mean or median as fill fn.")

        # Check if there are NaN values remaining.
        assert opt_df.isnull().to_numpy().sum() == 0, "Not all NaN values of opt_df have been dropped."
        assert gu2020_df.isnull().to_numpy().sum() == 0, "Not all NaN values of gu2020_df have been filled."
        print("Done!")

    print(f"There are now {gu2020_df.isnull().to_numpy().sum()} missing values.")
    # Merge option data with underlying Gu2020 stock data.
    # "inner" removes about 2000 rows. "left" would have these as NaNs.
    print("Merge option data with underlying Gu2020 stock data...")
    final_df = opt_df.merge(gu2020_df, on=["date", "permno"], how="inner")
    print(f"Done! {len(opt_df)-len(final_df)} rows have been removed by the "
          "inner join of the option and stock data on 'date' and 'permno'.")
    # Put option returns at the end.
    opt_ret = final_df.pop("option_ret")
    final_df["option_ret"] = opt_ret

    # Transform Categorical variables to OneHot-Numeric array
    print("Transform categorical variables cp_flag and sic2 code to OneHot-numeric array "
        "and drop useless columns.")
    onehotencoder = make_column_transformer((OneHotEncoder(), 
                                            ["cp_flag", "sic2"]), remainder="drop")
    transformed = onehotencoder.fit_transform(final_df)
    transformed_df = pd.DataFrame(transformed.toarray(), 
                                    columns=onehotencoder.transformers_[0][1]
                                    .get_feature_names_out(["cp_flag", "sic2"]))
    # Drop old columns cp_flag, sic2.
    final_df = final_df.drop(["cp_flag", "sic2"], axis=1)
    # Concatenate both dataframes (memory intensive?)
    final_df = pd.concat([final_df, transformed_df], axis=1)
    # Sort values by date! (secondly by secid -> but better to shuffle within date later?)
    final_df = final_df.sort_values(["date", "secid"]).reset_index(drop=True)
    # Remove columns useless for data analysis.
    final_df = final_df.drop(["secid", "optionid", "exdate", "sdate", "edate", "permno", "index"], axis=1)
    print("Done!")

    # Save final_df depending on arguments. Final_df is equal to the 'big' dataset.
    print("Save final dataframe...")
    save_df(path, final_df, args)
    print("Done!")
    print("---")
    print("All done.")


def save_df(path: Path, final_df: pd.DataFrame, args):
    # Create and save small, medium and big datasets.
    # Small dataset takes the option_return data + the newly created one_hot columns 
    # + option return, gives in total 22 columns (incl. target, option_return)
    prefix = f"final_df_call_cao{args.tag}"
    if args.size == "small": # 22 columns.
        filename = prefix + "_small.parquet"
        # Only keep first 19 columns + 'cp_flag_C', 'cp_flag_P' and the target 'option_ret'
        small_df_columns = (final_df.iloc[:, :19].columns
                        .append(final_df.loc[:, ["cp_flag_C", "option_ret", "OPT_RET"]].columns)) ##MATHIAS REMOVE CP_FLAG_P
        small_final_df = final_df.loc[:, small_df_columns]
        small_final_df.to_parquet(path/filename) # 22 columns.
    
    elif args.size == "medium": # 116 columns.
        # Medium dataset takes all option + stock data but no sic onehot columns.
        if args.fill_na:  # NaNs filled.
            filename = prefix + f"_med_fill{args.fill_fn}.parquet"
        else: # NaNs not filled.
            filename = prefix + "_med_nofill.parquet"
        # Remove all sic codes from big dataset to get medium.
        medium_final_df = final_df.loc[:, ~final_df.columns.str.startswith('sic2')]
        medium_final_df.to_parquet(path/filename) # 116 columns.

    elif args.size == "big": # 179 columns.
        # Big dataset leaves data as it is.
        if args.fill_na: # NaNs filled.
            filename= prefix + f"_big_fill{args.fill_fn}.parquet"
        else: # NaNs not filled
            filename = prefix + "_big_nofill.parquet"
        final_df.to_parquet(path/filename) # 179 columns.

    print(f"The final dataframe with specifications:\nsize: {args.size},\nfill_na: {args.fill_na}, "
          f"\nfill_fn: {args.fill_fn}\nhas been saved to {path/filename} .")
    
def add_sic_manually(df):
    """Adds missing sic codes to all rows with the respective permno number."""
    df.loc[df["permno"] == 18312, "sic2"] = 28 #Moderna https://sec.report/CIK/0001682852
    df.loc[df["permno"] == 18428, "sic2"] = 28 #Dow Inc https://sec.report/Ticker/DOW
    df.loc[df["permno"] == 93429, "sic2"] = 62 #CBOE Global Markets https://sec.report/Ticker/CBOE
    df.loc[df["permno"] == 18143, "sic2"] = 28 #Linde https://sec.report/CIK/0001707925
    df.loc[df["permno"] == 19285, "sic2"] = 35 #Carrier Global Corp https://sec.report/Ticker/CARR
    df.loc[df["permno"] == 18592, "sic2"] = 1 #Corteva https://sec.report/CIK/0001755672
    df.loc[df["permno"] == 17942, "sic2"] = 20 #Keurig Dr Pepper https://sec.report/Ticker/kdp
    df.loc[df["permno"] == 18420, "sic2"] = 48 #FOX Corp https://sec.report/Ticker/foxa
    df.loc[df["permno"] == 20057, "sic2"] = 28 #Viatris https://sec.report/Ticker/foxa
    df.loc[df["permno"] == 15850, "sic2"] = 72 #Match Group https://sec.report/Ticker/mtch
    df.loc[df["permno"] == 32791, "sic2"] = 35 #Weatherford https://sec.report/Ticker/wft
    df.loc[df["permno"] == 19286, "sic2"] = 36 #OTIS https://sec.report/Ticker/OTIS
    df.loc[df["permno"] == 17700, "sic2"] = 73 #Ceridian https://sec.report/Ticker/CDAY
    df.loc[df["permno"] == 18724, "sic2"] = 39 #Amcor https://sec.report/Ticker/AMCR
    df.loc[df["permno"] == 38850, "sic2"] = 24 #Skyline https://sec.report/Ticker/sky
    df.loc[df["permno"] == 19807, "sic2"] = 38 #Vontier https://sec.report/Ticker/VNT
    df.loc[df["permno"] == 78840, "sic2"] = 48 #IAC Interactive Corp ->sp500 const. list
    # ----
    df.loc[df["permno"] == 81594, "sic2"] = 13 #Dynegy, only 8 were NaN, rest was =13
    df.loc[df["permno"] == 88601, "sic2"] = 49 #Mirant, first 4 were NaN, rest =49
    df.loc[df["permno"] == 80913, "sic2"] = 73 #ACS, last month NaN, rest =73
    df.loc[df["permno"] == 44652, "sic2"] = 54 #American Stores Company, last month NaN, rest =54
    df.loc[df["permno"] == 76563, "sic2"] = 54 #Meyer Fred Inc., last month NaN, rest =54

    return df

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer

In [ ]:
# prepare_dataset(args)

In [22]:
path = r"C:\Users\Mathiass\OneDrive - Universität Zürich UZH\Documents\mt_literature\data"
path = Path(path)

In [23]:
a = pd.read_parquet(path/"final_df_call_cao_small.parquet")

In [24]:
path = r"C:\Users\Mathiass\OneDrive - Universität Zürich UZH\Documents\mt_literature\data\old_optionreturn"
path = Path(path)

In [25]:
b = pd.read_parquet(path/"final_df_small.parquet")

In [28]:
len(a["date"].unique())

310

In [29]:
len(b["date"].unique())

311

In [30]:
26 * 12 - 1

311

In [19]:
b = sorted(list(set((a["date"].apply(lambda x: x.strftime("%Y"))).tolist())))

In [21]:
for i in b:
    print(i, len(a[a["date"] < i]))

1996 0
1997 9258
1998 21515
1999 37357
2000 55035
2001 78049
2002 94953
2003 109016
2004 125524
2005 145013
2006 169906
2007 198824
2008 234731
2009 266950
2010 295124
2011 313814
2012 340628
2013 362546
2014 385956
2015 419568
2016 451661
2017 481534
2018 519621
2019 565885
2020 608524
2021 692814


In [ ]:
a

In [ ]:
path = r"C:\Users\Mathiass\Documents\Projects\master-thesis\data"
path = Path(path)

In [ ]:
c = pd.read_parquet(path/"final_df_call_cao_small.parquet")

In [ ]:
c

In [ ]:
(a == c).all().all()

In [ ]:
a

In [ ]:
c

In [ ]:
from pandas.testing import assert_frame_equal

In [ ]:
assert_frame_equal(a, c)

In [ ]:
path = r"C:\Users\Mathiass\Documents\Projects\master-thesis\data"
path = Path(path)

In [ ]:
d = pd.read_parquet(path/"final_df_call_cao_small1.parquet")

In [ ]:
e = pd.read_parquet(path/"final_df_call_cao_med_fillmean1.parquet")

In [ ]:
f = pd.read_parquet(path/"final_df_call_cao_big_fillmean1.parquet")

In [ ]:
g = pd.read_parquet(path/"final_df_call_cao_small.parquet")

In [ ]:
h = pd.read_parquet(path/"final_df_call_cao_med_fillmean.parquet")

In [ ]:
i = pd.read_parquet(path/"final_df_call_cao_big_fillmean.parquet")

In [ ]:
(g == .drop(columns="cp_flag_C")).all().all()

In [ ]:
(h == e.drop(columns="cp_flag_C")).all().all()

In [ ]:
(i == f.drop(columns="cp_flag_C")).all().all()

In [ ]:
assert_frame_equal(g, c.drop(columns="cp_flag_C"))

In [ ]:
assert_frame_equal(h, e.drop(columns="cp_flag_C"))

In [ ]:
assert_frame_equal(i, f.drop(columns="cp_flag_C"))

In [ ]:
print(len(raw) - len(raw_call), "Put rows have been deleted.")

In [ ]:
raw_call[raw_call["optionid"] == 10564133]

In [ ]:
a = raw_call[raw_call["option_ret"].isnull()]

In [ ]:
a["optionid"].head(34)

In [ ]:
for i, opt_id in enumerate(a["optionid"]):
    print("index:", i, "length:", len(raw_call[raw_call["optionid"] == opt_id]))

In [ ]:
raw_call[raw_call["optionid"] == 10564133]

In [ ]:
raw_call[raw_call["optionid"] == 10564133].iloc[0, :]["date"] + MonthEnd(1) == raw_call[raw_call["optionid"] == 10564133].iloc[1, :]["date"]

In [ ]:
raw_call[raw_call["optionid"] == 10564133].iloc[0, :]["date"] + MonthEnd(1)

In [ ]:
raw_call[raw_call["optionid"] == 10564133].iloc[1, :]["date"]

In [ ]:
raw_call[raw_call["secid"] == 5061]

In [ ]:
optionids = raw_call["optionid"].unique()

In [ ]:
optionids

In [ ]:
def count_succ(dates):
    counter = 0
    current = pd.to_datetime("1900") #startdate outside of dataset
    for date in dates:
        if date == current + MonthEnd(1):
            counter += 1
        current = date
    return counter

In [ ]:
raw_call["date"].iloc[0]

In [ ]:
raw_call

In [ ]:
raw_call.groupby("optionid")["date"]

In [ ]:
# raw_call.groupby("optionid")["date"].agg(count_succ)

In [ ]:
raw_call["optionid"].unique()

In [ ]:
optid_counts = raw_call.groupby("optionid")["secid"].agg("count")

In [ ]:
optid_counts[10980421]

In [ ]:
# only optionids with count > 1 can potentially have a return
atleasttwo = optid_counts[optid_counts > 1].index.tolist()

In [ ]:
np.isin(10980421, atleasttwo)

In [ ]:
# select only rows that have more than 1 options per optionid
raw_call_2 = raw_call[np.isin(raw_call["optionid"], atleasttwo)]

In [ ]:
raw_call_2

In [ ]:
import time
st_time = time.time()
res = raw_call_2.groupby("optionid")["date"].agg(count_succ)
e_time = time.time()
print(e_time - st_time, "seconds")

In [ ]:
optionids

In [ ]:
len(optionids)

In [ ]:
res

### Number of returns that can be calculated:

In [ ]:
# Number of returns that can be used as dataset:
sum(res[res > 0])

In [ ]:
res[res > 0]

In [ ]:
atleast1succ = res[res > 0].index.tolist()

In [ ]:
# raw_call_final = raw_call[np.isin(raw_call["optionid"], atleast1succ)]

In [ ]:
def opt_ret(df):
    t1 = (df["delta"] * df["spotprice"].shift(-1) - df["mid_price"].shift(-1))
    t = (df["delta"] * df["spotprice"] - df["mid_price"])
    return t1/t - 1

In [ ]:
raw_call[raw_call["optionid"] == 10914516].index

In [ ]:
raw_call[raw_call["optionid"] == 10914516]

In [ ]:
(0.657941 * 29.875 - 2.1875) / (0.657941 * 31.750 - 3.5625) - 1

In [ ]:
(0.546475 * 33.125 - 3.5625 ) / (0.546475 * 29.875 - 2.1875) - 1

In [ ]:
opt = opt_ret(raw_call[raw_call["optionid"] == 10914516])
raw_call.loc[opt.index, "option_ret2"] = opt

In [ ]:
raw_call[raw_call["optionid"] == 10914516].iloc[1]["date"]

In [ ]:
# opt_ret2(raw_call)

In [ ]:
raw_call[raw_call["optionid"] == 10914516]

In [ ]:
d = atleast1succ[:10000]

In [ ]:
for opt_id in tqdm(d):
    opt_return = opt_ret(raw_call[raw_call["optionid"] == opt_id])
    raw_call.loc[opt_return.index, "option_ret2"] = opt_return

In [ ]:
raw_call

In [ ]:
def opt_ret2(df, atleast1succ):
    for opt_id in tqdm(atleast1succ):
        df_opt = df[df["optionid"] == opt_id]
        for i, idx in enumerate(df_opt.index):
            if i + 1 == len(df_opt.index):
                df.loc[idx, "OPT_RET"] = None
                break
            if df_opt.iloc[i]["date"] + MonthEnd(1) == df_opt.iloc[i+1]["date"]:
                t1 = (df_opt.iloc[i]["delta"] * df_opt.iloc[i+1]["spotprice"]) - df_opt.iloc[i+1]["mid_price"]
                t = (df_opt.iloc[i]["delta"]  * df_opt.iloc[i]["spotprice"]) - df_opt.iloc[i]["mid_price"]
                df.loc[idx, "OPT_RET"] = t1/t - 1
            else:
                df.loc[idx, "OPT_RET"] = None
    return df

In [ ]:
e = raw_call[np.isin(raw_call["optionid"], d)]

In [ ]:
# Takes 33mins
raw_call_filled = opt_ret2(raw_call, atleast1succ)

In [ ]:
raw_call_filled["OPT_RET"].count()

In [ ]:
raw_call_filled.optionid

In [ ]:
opt_ret_count = raw_call_filled.groupby("optionid")["OPT_RET"].agg("count")

In [ ]:
opt_ret_count[opt_ret_count==1]

In [ ]:
res[res == 1]

In [ ]:
rem = set(opt_ret_count[opt_ret_count==1].index.tolist()) - set(res[res == 1].index.tolist())

In [ ]:
for i in rem:
    print(i in atleast1succ)

In [ ]:
opt_ret2(raw_call_filled[raw_call_filled["optionid"] == 10167707], [10167707])

In [ ]:
opt_ret2(raw_call[raw_call["optionid"] == 10167707], [10167707])

In [ ]:
opt_ret2(raw_call[raw_call["optionid"] == 10167707].iloc[:, :-1], atleast1succ)

In [ ]:
raw_call_filled[raw_call_filled["optionid"] == 10167707]["delta"]

In [ ]:
raw_call_filled[raw_call_filled["optionid"] == 10167707].shift(-1)

In [ ]:
(0.341984 * 27.875 - 1.71875) / (0.341984 * 24.500 - 1.56250) - 1

In [ ]:
optid_counts

In [ ]:
def count_na(x):
    return x.isna().sum()

In [ ]:
#count na per optionid
na_counts = raw_call_filled.groupby("optionid")["OPT_RET"].agg(count_na)

In [ ]:
a = optid_counts - na_counts

In [ ]:
raw_call[raw_call["optionid"] == 123889082]["delta"]

In [ ]:
raw_call[raw_call["optionid"] == 123889082]

In [ ]:
(0.421645*0.9860 - 0.150) / (0.421645 * 0.3345 - 0.100) - 1

In [ ]:
#gamestop
raw_call[raw_call["optionid"] == 137759524]

In [ ]:
raw_call.drop(["option_ret2"], axis=1).dropna()["OPT_RET"].sort_values(ascending=False)